In [2]:
import os

In [3]:
%pwd

'c:\\Users\\B HEMANTH\\OneDrive\\Desktop\\e2e_text_summary\\end2end_text_summarization\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\B HEMANTH\\OneDrive\\Desktop\\e2e_text_summary\\end2end_text_summarization'

In [37]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()

CONFIG_FILE_PATH = PROJECT_ROOT / "config" / "config.yaml"
PARAMS_FILE_PATH = PROJECT_ROOT / "params.yaml"

print(CONFIG_FILE_PATH)
print(CONFIG_FILE_PATH.exists())

c:\Users\B HEMANTH\OneDrive\Desktop\e2e_text_summary\end2end_text_summarization\config\config.yaml
True


In [42]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
# from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories


In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [31]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size


In [48]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [33]:
from pathlib import Path

path = Path("../config/config.yaml").exists()
print(path)

False


c:\Users\B HEMANTH\OneDrive\Desktop\e2e_text_summary\end2end_text_summarization\config\config.yaml
True


In [49]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-02-27 00:54:24,134: INFO: common: yaml file: c:\Users\B HEMANTH\OneDrive\Desktop\e2e_text_summary\end2end_text_summarization\config\config.yaml loaded successfuly]
[2026-02-27 00:54:24,136: INFO: common: yaml file: c:\Users\B HEMANTH\OneDrive\Desktop\e2e_text_summary\end2end_text_summarization\params.yaml loaded successfuly]
[2026-02-27 00:54:24,138: INFO: common: created directory at: artifacts]
[2026-02-27 00:54:24,140: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-27 00:54:27,492: INFO: 499299850: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 8377572
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3aea79eaa36efc9339c42d3a61413f75f62f7e5676002d7473efbe842605ace6"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Reques